## 第一部分数据预处理
流程如下：
####  1. 导入必要的包并读取数据 
####  2. 对两个表分别进行预处理，预处理流程如下：
##### 对c1表：去除卡号相同的重复值、采用填充法进行缺失值处理、异常值检测
#####  对c2表：去除重复值、缺失值检测、异常值处理（如销售金额为负数、购买数量为负数等）
#### 3. 合并两个表便于后续处理，连接方式为外连接

In [1]:
# -*- coding: utf-8 -*-
"""
Created on 2020-07-19

@author: 豆奶
"""
# 导入必要的包
import pandas as pd

## 对c1表的预处理

In [2]:
data1 = pd.read_excel("../data/cumcm2018c1.xlsx")
data1.head()

,kh,csrq,xb,djsj
0,c68b20b4,2002-11-02 00:00:00,0.0,2013-05-11 00:00:00.000
1,1ca15332,NaN,0.0,2004-11-04 16:31:52.436
2,a37cc182,1967-02-17 00:00:00,0.0,2004-12-31 21:24:34.216
3,2ab88539,1982-06-01 00:00:00,0.0,2010-11-19 00:00:00.000
4,b4c77269,1964-02-05 00:00:00,0.0,2007-12-14 00:00:00.000


In [3]:
# 去除卡号重复值
print("去除重复值前的数据量", data1.shape)   
data1.drop_duplicates(subset=['kh'],keep='first',inplace=True)
print("去除重复值后的数据量", data1.shape)  

去除重复值前的数据量 (194760, 4)
去除重复值后的数据量 (194754, 4)


In [4]:
# 缺失值处理
print('处理前缺失值数目是：\n', data1.isnull().sum())  # 可以看到缺失值较多，采取填充法进行缺失值处理
data1 = data1.fillna(axis=0,method='ffill')   # 填充法
print('处理后缺失值数目是：\n', data1.isnull().sum())

处理前缺失值数目是：
 kh          0
csrq    34187
xb       9435
djsj    12684
dtype: int64
处理后缺失值数目是：
 kh      0
csrq    0
xb      0
djsj    0
dtype: int64


## 对c2表的预处理

In [6]:
data2 = pd.read_csv("../data/cumcm2018c2.csv")
data2.head()

d:\python3.7.5\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,kh,dtime,spbm,sl,sj,je,spmc,jf,syjh,djh,gzbm,gzmc
0,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,1,290.0,270.20,兰芝化妆品正价瓶,270.20,6,25bb,8077.0,兰芝柜
1,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,1,325.0,302.80,兰芝化妆品正价瓶,302.80,6,25bb,8077.0,兰芝柜
2,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,1,195.0,181.80,兰芝化妆品正价瓶,181.80,6,25bb,8077.0,兰芝柜
3,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,1,270.0,251.55,兰芝化妆品正价瓶,251.55,6,25bb,8077.0,兰芝柜
4,1be1e3fe,2015-01-01 00:05:41.593,f09c9303,2,245.0,456.55,兰芝化妆品正价瓶,456.55,6,25bb,8077.0,兰芝柜


In [7]:
# 去除重复值
print("去除重复值前的数据量", data2.shape)  
data2.drop_duplicates(subset=None,keep='first',inplace=True)
print("去除重复值后的数据量", data2.shape)  

去除重复值前的数据量 (1893532, 12)
去除重复值后的数据量 (1893532, 12)


In [9]:
# 缺失值检测
print('处理前缺失值数目是：\n', data2.isnull().sum())  
# 可以看到缺失值部分为：会员卡号，积分，柜组编码，柜组名，因此不需要进行填充

处理前缺失值数目是：
 kh       1017486
dtime          0
spbm           0
sl             0
sj             0
je             0
spmc           0
jf       1017486
syjh           0
djh            0
gzbm     1017486
gzmc     1036956
dtype: int64


In [10]:
# 异常值检测
print(data2.min(), '\n')
print(data2.max())

sl        -381.0
sj           0.1
je     -671257.5
jf     -201616.0
syjh         1.0
gzbm         0.0
dtype: float64 

sl         1492.0
sj      1342515.0
je      1342515.0
jf       268503.0
syjh        320.0
gzbm       9005.0
dtype: float64


In [11]:
# 异常值处理，对于异常值的阈值，可与商场负责人进行确认，这里暂时凭个人感觉进行确定
print("去除异常值前的数据量", data2.shape)  

data2 = data2.drop(data2[(data2.je< 0)].index)  # 去除销售金额为负数的数据
data2 = data2.drop(data2[(data2.je>5000)].index) # 去除销售金额过大的数据，这里暂时定为5000，具体可与商场进行商讨

data2 = data2.drop(data2[(data2.sj< 0.5)].index)  # 去除售价为负数的数据
data2 = data2.drop(data2[(data2.sj>5000)].index) # 去除售价过大的数据，这里暂时定为5000

data2 = data2.drop(data2[(data2.sl< 0)].index)  # 去除购买数量为负数的数据
data2 = data2.drop(data2[(data2.sl> 1000)].index)  # 去除购买数量过大的数据

data2 = data2.drop(data2[(data2.jf< 0)].index)  # 去除积分为负数的数据

# 处理后的结果查看
print("去除异常值后的数据量", data2.shape)  
print(data2.min(), '\n')
print(data2.max())

去除异常值前的数据量 (1893532, 12)
去除异常值后的数据量 (1823181, 12)
sl      1.0
sj      1.0
je      0.0
jf      0.0
syjh    1.0
gzbm    0.0
dtype: float64 

sl        200.0
sj       5000.0
je       5000.0
jf      14250.0
syjh      320.0
gzbm     9005.0
dtype: float64


## 合并两张表

In [16]:
result = pd.merge(data1, data2, on='kh', how="outer")

In [17]:
print(result.shape) 
result.head()

(1971886, 15)


,kh,csrq,xb,djsj,dtime,spbm,sl,sj,je,spmc,jf,syjh,djh,gzbm,gzmc
0,c68b20b4,2002-11-02 00:00:00,0.0,2013-05-11 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1ca15332,2002-11-02 00:00:00,0.0,2004-11-04 16:31:52.436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a37cc182,1967-02-17 00:00:00,0.0,2004-12-31 21:24:34.216,2016-09-25 10:49:14.016,d62a69e3,1.0,499.0,297.07,WMF D无,297.08,101.0,7cd8,7296.0,WMF 柜
3,a37cc182,1967-02-17 00:00:00,0.0,2004-12-31 21:24:34.216,2017-02-28 17:17:35.533,252403ef,1.0,598.0,598.00,双立人商品 F无,598.00,102.0,ed0a,7242.0,双立人柜
4,2ab88539,1982-06-01 00:00:00,0.0,2010-11-19 00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
result.to_csv('../data/task1.csv',  encoding='utf-8')